In [2]:
!pip install nltk spacy
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 435.7 kB/s eta 0:00:30
     --------------------------------------- 0.1/12.8 MB 744.7 kB/s eta 0:00:18
     --------------------------------------- 0.1/12.8 MB 654.9 kB/s eta 0:00:20
      --------------------------------------- 0.2/12.8 MB 1.1 MB/s eta 0:00:12
      --------------------------------------- 0.3/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.4/12.8 MB 1.2 MB/s eta 0:00:10
     - -------------------------------------- 0.6/12.8 MB 1.7 MB/s eta 0:00:08
     -- ------------------------------------- 0.8/12.8 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.9/12.8 MB 2.0 MB/s eta 0:00:06
     --- ------------------------------------ 1.1/12.8 MB 2.2 MB/s eta 0:00:06
     ---- ----------------------------------- 1.3/12.8 MB 2.3 M

In [7]:
pip install transformers torch


Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Sample FAQs (You should replace this with your actual FAQs)
faq_data = [
    ("What is the return policy?", "Our return policy lasts 30 days."),
    ("How do I track my order?", "You can track your order using the tracking number provided in your confirmation email."),
    ("What payment methods are accepted?", "We accept Visa, MasterCard, and PayPal."),
    ("How do I contact customer support?", "You can contact customer support via email or phone."),
]

questions, answers = zip(*faq_data)

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to encode text using BERT
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Encode all questions
encoded_questions = np.vstack([encode_text(q) for q in questions])


In [6]:
def get_response(user_query):
    # Encode the user query
    encoded_query = encode_text(user_query)
    
    # Compute cosine similarity
    similarities = cosine_similarity(encoded_query, encoded_questions).flatten()
    most_similar_index = np.argmax(similarities)
    
    # Threshold for similarity
    if similarities[most_similar_index] < 0.2:
        return "I'm sorry, I don't understand your question. Can you please rephrase?"
    else:
        return answers[most_similar_index]

# Example Usage
user_query = "How can I track my shipment?"
response = get_response(user_query)
print(response)  # Output: You can track your order using the tracking number provided in your confirmation email.


You can track your order using the tracking number provided in your confirmation email.


In [7]:
feedback_data = []

def get_response_with_feedback(user_query):
    response = get_response(user_query)
    print(response)
    feedback = input("Was this response helpful? (yes/no): ")
    feedback_data.append((user_query, response, feedback))
    return response

# Example Usage
user_query = "How can I track my shipment?"
response = get_response_with_feedback(user_query)


You can track your order using the tracking number provided in your confirmation email.
